- Black-Litterman Model Integration:
Use the forecasts from the ML models (e.g., LSTM predicted returns) as “views” in the Black-Litterman (BL) framework to derive a new implied return vector and construct a posterior distribution of returns. We will then solve for the BL-implied optimal portfolio weights.
- Reinforcement Learning for Dynamic Rebalancing:
Create a reinforcement learning (RL) environment that simulates a portfolio rebalancing scenario. The RL agent will learn to adjust portfolio weights over time to maximize a performance metric (like risk-adjusted returns). Transaction costs will be included in the reward function to encourage cost-effective trading.
- Risk Parity Approach:
Implement a simple risk parity portfolio solution as an alternative or baseline approach.
- Transaction Costs:
Incorporate transaction costs into both the BL optimization (as part of the optimization constraints or objective) and the RL environment’s reward structure.

# Black-Litterman Equations Used

## Given:

* Prior expected returns vector: μ (equilibrium returns)
* Covariance matrix: Σ
* Views matrix: P, views vector: Q
* Uncertainty parameter: τ
* View uncertainty matrix: Ω

## The Black-Litterman posterior is:

μ<sub>post</sub> = μ + τΣP<sup>T</sup>(PτΣP<sup>T</sup> + Ω)<sup>-1</sup>(Q - Pμ)

Σ<sub>post</sub> = Σ + ΣP<sup>T</sup>(PτΣP<sup>T</sup> + Ω)<sup>-1</sup>PτΣ

##  Setup and Data Preparation

In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

def fetch_data(tickers, start_date, end_date):
    """
    Fetch historical price data for given tickers.
    :param tickers: List of stock tickers.
    :param start_date: Start date for the historical data.
    :param end_date: End date for the historical data.
    :return: DataFrame with adjusted closing prices.
    """
    
    data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
    return data

tickers = ["BWXT", "CEG", "NEE", "VRT"]  # Add 'SWEC-B.ST' if supported
data = yf.download(tickers, start="2022-02-01", end="2024-12-05")['Adj Close']

print(data.head())

In [ ]:
# Compute daily returns
returns = data.pct_change().dropna()
print(returns.head())

In [ ]:
# Train test split
train_size = int(len(returns) * 0.83)
train_returns = returns.iloc[:train_size]
test_returns = returns.iloc[train_size:]

print("Train set length:", len(train_returns))
print("Test set length:", len(test_returns))

In [ ]:
import ta

price_data = data.copy()  # Just renaming for clarity
# Example: Compute RSI for each ticker
for ticker in price_data.columns:
    price_data[f"{ticker}_RSI"] = ta.momentum.RSIIndicator(price_data[ticker], window=14).rsi()

# Drop rows with NaN caused by indicator computation
price_data.dropna(inplace=True)

print(price_data.head())

In [ ]:
# Example: 5-day ahead returns as target
horizon = 5
for ticker in data.columns:
    price_data[f"{ticker}_future_return"] = data[ticker].pct_change(horizon).shift(-horizon)

# Drop rows with NaN in the future returns
price_data.dropna(inplace=True)

print(price_data.head())

## Compute Prior (Market Equilibrium) Parameters

We use historical mean returns and covariance from the training set as the “prior”:

In [ ]:
import numpy as np

# Find rows with inf or very large values
problematic_rows = train_returns[(np.isinf(train_returns)) | (train_returns > 1e5) | (train_returns < -1e5)]
print(problematic_rows)

In [ ]:
import numpy as np

print("NaN values per column:")
print(train_returns.isna().sum())

print("Number of Infinite values:")
print(np.isinf(train_returns).sum())

In [ ]:
# Find rows with infinite or extremely large values
bad_rows = train_returns[(np.isinf(train_returns)) | (train_returns.abs() > 1e5)]
print("Problematic rows:\n", bad_rows)

In [ ]:
train_returns = train_returns.replace([np.inf, -np.inf], np.nan)
train_returns = train_returns.dropna(how='any')  # Drop rows with NaNs

In [ ]:
# When computing returns:
returns = price_data.pct_change().replace([np.inf, -np.inf], np.nan).dropna(how='any')

In [ ]:
# Suppose 'VRT' is problematic:
train_returns = train_returns.drop(columns=['VRT'], errors='ignore')

In [ ]:
# Remove days where prices are zero or negative (if any):
price_data = price_data[price_data > 0].dropna(how='any')
returns = price_data.pct_change().dropna()
# Then create train_returns again and retry

In [ ]:
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage

# Prior: Historical mean returns & covariance
mu = mean_historical_return(train_returns)    # vector of prior expected returns
S = CovarianceShrinkage(train_returns).ledoit_wolf()  # covariance matrix

In [ ]:
mu = mean_historical_return(train_returns)
S = CovarianceShrinkage(train_returns).ledoit_wolf()

## Construct Views from ML Predictions

In [ ]:
# Mock ML predicted returns (views):
view_returns = pd.Series([0.001, 0.0015, 0.0005], index=mu.index)  

## Form Views (P, Q, Omega):
	•	We have 3 views: one for each asset’s return. P is the identity matrix since we have direct views on each asset’s return level.
	•	Q is the vector of view returns.
	•	Omega represents the uncertainty in views. We choose it as a small diagonal matrix.

In [ ]:
assets = mu.index.tolist()
n = len(assets)
P = np.eye(n)
Q = view_returns.values
tau = 0.05

# Omega: If we assume we trust our views moderately, set Omega proportional to diag of τΣ
Omega = np.diag(np.diag(P @ (tau * S.values) @ P.T))  # same dimension as P @ S @ P^T

## Black-Litterman Posterior Calculation (Manual)

In [ ]:
# Compute posterior mean and covariance
Sigma = S.values
mu_prior = mu.values.reshape(-1,1)

A = np.linalg.inv(P @ (tau * Sigma) @ P.T + Omega)
mu_post = mu_prior + tau * Sigma @ P.T @ A @ (Q - P @ mu_prior.flatten())

Sigma_post = S.values + (tau * Sigma) - (tau * Sigma @ P.T @ A @ P @ (tau * Sigma))

mu_post = pd.Series(mu_post.flatten(), index=mu.index)
Sigma_post = pd.DataFrame(Sigma_post, index=mu.index, columns=mu.index)

print("Posterior Mean Returns:\n", mu_post)
print("Posterior Covariance:\n", Sigma_post)

## Optimize Portfolio with Transaction Costs (Static Optimization)

Assume we have a current portfolio and want to find a new allocation that maximizes expected returns minus risk penalties and transaction costs.

In [ ]:
# Suppose current weights:
w_current = np.array([0.4, 0.4, 0.2])

w = cp.Variable(n)
expected_return = mu_post.values @ w
risk_aversion = 10
lambda_tc = 0.001
portfolio_var = cp.quad_form(w, Sigma_post.values)
transaction_costs = lambda_tc * cp.norm(w - w_current, 1)

prob = cp.Problem(cp.Maximize(expected_return - risk_aversion*portfolio_var - transaction_costs),
                  [cp.sum(w) == 1,
                   w >= 0])
prob.solve()

w_opt = w.value
print("Optimized portfolio weights from Black-Litterman with TC:", w_opt)

## Risk Parity Approach

Implement a simple risk parity solution as a baseline.

In [ ]:
def risk_contributions(weights, cov):
    w = weights.reshape(-1,1)
    total_variance = (w.T @ cov @ w).item()
    mc = cov @ w
    rc = (w * mc) / total_variance
    return rc.flatten()

def risk_parity(cov, tol=1e-6, max_iter=1000):
    n = cov.shape[0]
    w = np.ones(n)/n
    for _ in range(max_iter):
        rc = risk_contributions(w, cov)
        # We want all RC equal, target = 1/n
        diff = rc - 1/n
        if np.linalg.norm(diff) < tol:
            break
        # Gradient step:
        w = w - 0.01 * diff
        w = np.clip(w, 0, None)
        w = w / w.sum()
    return w

rp_weights = risk_parity(Sigma_post.values)
print("Risk parity weights:", rp_weights)

##  Reinforcement Learning for Dynamic Rebalancing

We create a custom environment where the agent decides how to rebalance daily. The observation includes predicted returns and current weights, and the reward is the daily portfolio return minus transaction costs. The agent learns to rebalance over time.

Note: This is a simplified environment. In practice, you’d incorporate more complex features (volatility forecasts, regimes, etc.), and train for many timesteps. Also ensure stable-baselines3 and gymnasium are installed.

In [ ]:
!pip install "gymnasium[classic_control]"  # Example, depends on environment
from stable_baselines3 import PPO
import gymnasium as gym
from gymnasium import spaces

class PortfolioEnv(gym.Env):
    metadata = {'render_modes': ['human']}
    
    def __init__(self, price_data, predicted_returns, initial_capital=100000, transaction_cost=0.001):
        super(PortfolioEnv, self).__init__()
        
        self.price_data = price_data
        self.predicted_returns = predicted_returns
        self.initial_capital = initial_capital
        self.transaction_cost = transaction_cost
        
        self.assets = price_data.columns.tolist()
        self.n_assets = len(self.assets)
        
        # Observation: predicted returns + current weights
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(2*self.n_assets,), dtype=np.float32)
        
        # Action: change weights (-0.1 to 0.1)
        self.action_space = spaces.Box(low=-0.1, high=0.1, shape=(self.n_assets,), dtype=np.float32)
        
        self.current_step = 0
        self.current_weights = np.ones(self.n_assets)/self.n_assets
        self.portfolio_value = self.initial_capital
        
        # Precompute asset returns
        self.asset_returns = self.price_data.pct_change().fillna(0).values
    
    def _get_observation(self):
        obs = np.concatenate([self.predicted_returns[self.current_step], self.current_weights])
        return obs.astype(np.float32)
    
    def step(self, action):
        # Apply action to weights
        new_weights = self.current_weights + action
        new_weights = np.clip(new_weights, 0, None)
        if new_weights.sum() == 0:
            new_weights = np.ones(self.n_assets)/self.n_assets
        else:
            new_weights = new_weights / new_weights.sum()
        
        # Transaction costs
        tc_cost = self.transaction_cost * np.sum(np.abs(new_weights - self.current_weights)) * self.portfolio_value
        
        # Portfolio return
        asset_r = self.asset_returns[self.current_step]
        port_ret = (self.current_weights @ asset_r)
        
        self.portfolio_value = self.portfolio_value * (1 + port_ret) - tc_cost
        
        # Reward: daily return minus TC as a simple metric
        reward = port_ret - (tc_cost / self.portfolio_value)
        
        self.current_weights = new_weights
        self.current_step += 1
        done = (self.current_step >= len(self.asset_returns)-1)
        
        obs = self._get_observation()
        info = {'portfolio_value': self.portfolio_value}
        return obs, reward, done, False, info
    
    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0
        self.current_weights = np.ones(self.n_assets)/self.n_assets
        self.portfolio_value = self.initial_capital
        return self._get_observation(), {}
    
    def render(self):
        print(f"Step: {self.current_step}, PV: {self.portfolio_value}, Weights: {self.current_weights}")


# Mock predictions for RL environment (just repeat the BL posterior mean)
pred_array = np.tile(mu_post.values, (len(price_data), 1))

env = PortfolioEnv(price_data=price_data, predicted_returns=pred_array, transaction_cost=0.001)
model = PPO("MlpPolicy", env, verbose=0)
model.learn(total_timesteps=2000)

# Test the trained model
obs, _ = env.reset()
for _ in range(100):
    action, _states = model.predict(obs)
    obs, reward, done, truncated, info = env.step(action)
    if done or truncated:
        break

print("Final Portfolio Value after RL simulation:", info['portfolio_value'])

Summary

	•	Black-Litterman: Implemented manually with NumPy. No pybl package required.
	•	Transaction Costs: Incorporated into both BL optimization (static) and RL environment.
	•	Risk Parity: Provided as an alternative benchmark.
	•	Reinforcement Learning: Demonstrated using a simple PPO agent from stable-baselines3 with gymnasium.

This code is an example framework. In practice, you should:

	•	Integrate real ML predictions (e.g., from your LSTM model).
	•	Fine-tune RL hyperparameters and improve state and reward functions.
	•	Perform robust backtesting and validation.